In [6]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms
from torchvision.models.segmentation import deeplabv3_resnet50
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

In [7]:
# Custom Dataset Class
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        # Filter for image files only (e.g., .jpg, .png)
        self.images = [f for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx].replace('.jpg', '.png').replace('.png', '.png'))
        
        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")  # Convert to grayscale

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask)
        
        # Convert mask to binary (0 or 1)
        mask = (mask > 0.5).float()
        return image, mask

In [8]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [9]:
# Create datasets
full_dataset = SegmentationDataset(
    image_dir="../imagespng/imagespng",
    mask_dir="../binarypng/binarypng",
    transform=transform
)

In [10]:
# Split into training and validation sets (80% train, 20% val)
train_size = int(0.8 * len(full_dataset))
val_size = len(full_dataset) - train_size
train_dataset, val_dataset = random_split(full_dataset, [train_size, val_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=4, shuffle=False)

In [6]:
# Initialize model
model = deeplabv3_resnet50(pretrained=False, num_classes=1)  # Binary classification

/home/calviet/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/calviet/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [7]:
# Move the model to the appropriate device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [8]:
# Loss and optimizer
criterion = nn.BCEWithLogitsLoss()  # Binary cross-entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [9]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10):
    model.train()  # Set model to training mode
    for epoch in range(num_epochs):
        running_loss = 0.0
        for batch_idx, (images, masks) in enumerate(train_loader):
            images, masks = images.to(device), masks.to(device)  # Move to device
            optimizer.zero_grad()

            # Forward pass
            outputs = model(images)['out']
            outputs = outputs.squeeze(2)  # Remove the extra dimension (squeeze the 2nd dimension)

            # Compute the loss
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if batch_idx % 10 == 0:  # Print every 10 batches
                print(f"Epoch {epoch+1}/{num_epochs}, Batch {batch_idx+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

        avg_loss = running_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{num_epochs}, Avg Training Loss: {avg_loss:.4f}")

        # Validation step
        val_loss = evaluate(model, val_loader, criterion)
        print(f"Epoch {epoch+1}/{num_epochs}, Validation Loss: {val_loss:.4f}")

def evaluate(model, val_loader, criterion):
    model.eval()  # Set model to evaluation mode
    val_loss = 0.0
    with torch.no_grad():
        for images, masks in val_loader:
            images, masks = images.to(device), masks.to(device)  # Move to device
            outputs = model(images)['out']
            outputs = outputs.squeeze(2)  # Remove the extra dimension

            # Compute the loss
            loss = criterion(outputs, masks)
            val_loss += loss.item()

    return val_loss / len(val_loader)


In [10]:
# Start training
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=10)

Epoch 1/10, Batch 1/494, Loss: 0.7496
Epoch 1/10, Batch 11/494, Loss: 0.1745
Epoch 1/10, Batch 21/494, Loss: 0.0877
Epoch 1/10, Batch 31/494, Loss: 0.0645
Epoch 1/10, Batch 41/494, Loss: 0.0514
Epoch 1/10, Batch 51/494, Loss: 0.0506
Epoch 1/10, Batch 61/494, Loss: 0.0394
Epoch 1/10, Batch 71/494, Loss: 0.0357
Epoch 1/10, Batch 81/494, Loss: 0.0357
Epoch 1/10, Batch 91/494, Loss: 0.0350
Epoch 1/10, Batch 101/494, Loss: 0.0272
Epoch 1/10, Batch 111/494, Loss: 0.0309
Epoch 1/10, Batch 121/494, Loss: 0.0218
Epoch 1/10, Batch 131/494, Loss: 0.0313
Epoch 1/10, Batch 141/494, Loss: 0.0182
Epoch 1/10, Batch 151/494, Loss: 0.0207
Epoch 1/10, Batch 161/494, Loss: 0.0286
Epoch 1/10, Batch 171/494, Loss: 0.0262
Epoch 1/10, Batch 181/494, Loss: 0.0224
Epoch 1/10, Batch 191/494, Loss: 0.0208
Epoch 1/10, Batch 201/494, Loss: 0.0186
Epoch 1/10, Batch 211/494, Loss: 0.0172
Epoch 1/10, Batch 221/494, Loss: 0.0158
Epoch 1/10, Batch 231/494, Loss: 0.0249
Epoch 1/10, Batch 241/494, Loss: 0.0244
Epoch 1/10,

In [11]:
# Save the final model
torch.save(model.state_dict(), "deeplabv3_binary_segmentation_final.pth")

In [3]:
import torch
from torchvision import models
from torchvision.models.segmentation import deeplabv3_resnet50

# Load the model architecture
def get_model(num_classes=1):
    model = deeplabv3_resnet50(pretrained=False)
    model.classifier[4] = torch.nn.Conv2d(256, num_classes, kernel_size=1)
    return model

# Instantiate and load weights
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = get_model()
model.load_state_dict(torch.load('deeplabv3_binary_segmentation_final.pth', map_location=device))
model.to(device)



/home/calviet/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/calviet/.local/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_3811188/909945982.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. 

DeepLabV3(
  (backbone): IntermediateLayerGetter(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Se

In [15]:
from tqdm import tqdm
import torch.nn.functional as F

def test_model(model, dataloader, device='cuda'):
    model.eval()
    total_pixels = 0
    correct_pixels = 0
    intersection = 0
    union = 0
    dice_total = 0
    num_batches = 0

    with torch.no_grad():
        for images, masks in tqdm(dataloader, desc="Testing"):
            images, masks = images.to(device), masks.to(device)

            outputs = model(images)['out']
            preds = torch.sigmoid(outputs) > 0.5  # Boolean prediction

            preds = preds.squeeze(1).bool()
            masks = masks.squeeze(1).bool()

            correct_pixels += (preds == masks).sum().item()
            total_pixels += torch.numel(preds)

            inter = (preds & masks).sum().item()
            union_area = (preds | masks).sum().item()
            intersection += inter
            union += union_area

            # Dice score for this batch
            dice = (2 * inter) / (preds.sum().item() + masks.sum().item() + 1e-8)
            dice_total += dice
            num_batches += 1

    accuracy = correct_pixels / total_pixels
    iou = intersection / (union + 1e-8)
    dice_score = dice_total / num_batches

    print(f"\nAccuracy: {accuracy:.4f}")
    print(f"IoU: {iou:.4f}")
    print(f"Dice Score: {dice_score:.4f}")


In [16]:
# Now call the test function
test_model(model, val_loader, device=device)

Testing: 100%|██████████| 247/247 [02:35<00:00,  1.58it/s]


Accuracy: 0.9935
IoU: 0.9286
Dice Score: 0.9463
